In [1]:
%pip install tensorflow --user

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import *
word_vec_file="wiki-news-300d-1M.vec"

In [3]:
df=pd.read_csv("mp_csv.csv")
print(df.shape)
print(df.iloc[10])

(48, 4)
id                               11
tokens      add ins ise to calendar
slots     o task_name task_name o o
intent                      addTask
Name: 10, dtype: object


In [4]:
df

,id,tokens,slots,intent
0,1,hello,o,intial_welcome
1,2,hi there,o o,intial_welcome
2,3,hey,o,intial_welcome
3,4,hello there,o o,intial_welcome
4,5,good morning,o o,intial_welcome
5,6,good afternoon,o o,intial_welcome
6,7,good evening,o o,intial_welcome
7,8,my name is vinayak,o o o name,name
8,9,my name is sahil,o o o name,name
9,10,i am vinayak,o o name,name


In [5]:
n=df.to_numpy()
intents=[]
slots=[]
for row in n:
    for i,val in enumerate(row):
        if i==3:
            if val not in intents:
                intents.append(val)
        elif i==2:
            for slot in val.split():
                if slot not in slots:
                    slots.append(slot)
print(len(intents))
print(intents)
print(len(slots))
print(slots)

8
['intial_welcome', 'name', 'addTask', 'createSchedule', 'displaySchedule', 'removeTask', 'addExam', 'removeExam']
6
['o', 'name', 'task_name', 'task_deadline', 'exam', 'age']


In [6]:
import numpy as np
import io


def vocab(embf=None, n=100000, req=set()):
    if not embf:
        embf=word_vec_file
    def load_vectors(fname, n):
        fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
        # n, d = map(int, fin.readline().split())
        data = {}
        for i, line in enumerate(fin):
            if i == 0:
                continue            
            tokens = line.rstrip().split(' ')            
            if i<n:
                data[tokens[0]] = list(map(float, tokens[1:]))
            elif tokens[0] in req:
                data[tokens[0]] = list(map(float, tokens[1:]))
        return data

    word_to_vec = {}
    unk = [0] *300
    word_to_vec = load_vectors(embf, n)    
    return word_to_vec

In [7]:
def gen(train_set, intents, slots, word_to_vec):
    while True:        
        for row in train_set:            
            tokens=row[1].split()
            ip=np.zeros((1,len(tokens),300)) 
            for i, token in enumerate(tokens):
                if token in word_to_vec:
                    ip[0,i]=word_to_vec[token]
                else:
                    ip[0,i]=[0]*300
            op_intent=np.zeros((1, len(intents)))
            op_intent[0, intents.index(row[3])]=1
            op_slots=np.zeros((1,len(tokens),len(slots)))
            r_slots=row[2].split()
            if not len(r_slots) == len(tokens):
              print(tokens)
              print(r_slots)
              continue
            for i,slot in enumerate(r_slots):
                op_slots[0,i,slots.index(slot)]=1
                
            yield ip, (op_intent, op_slots)

In [10]:
word_to_vec=vocab(n=1*10**5)
g=gen(n, intents, slots, word_to_vec)
ip, op=g.__next__()
print(op[0])
print(op[1])

[[1. 0. 0. 0. 0. 0. 0. 0.]]
[[[1. 0. 0. 0. 0. 0.]]]


In [11]:
len(word_to_vec)

99999

In [12]:
n=df.to_numpy()
ip=Input(name="Input",batch_shape=(1,None,300))
enc_seq, enc_final_state=GRU(units=100, name="Encoder", return_sequences=True, return_state=True)(ip)
intent_classifier=Dense(units=len(intents), activation="softmax", name="intent_classifier")(enc_final_state)

decoder_seq=GRU(units=100 ,return_sequences=True, name="Decoder")(enc_seq, initial_state=enc_final_state)
slots_tagger=Dense(units=len(slots), activation="softmax", name="slots_tagger")(decoder_seq)
model=keras.models.Model(inputs=ip,outputs=(intent_classifier, slots_tagger),name="intent_and_slots")
model.summary()
# input()

# word_to_vec=vocab(n=1*10**5)
model.compile(optimizer=keras.optimizers.Adam(), loss="categorical_crossentropy", loss_weights=[1.0, 1.0], metrics=['accuracy'])
model.fit_generator(gen(n, intents, slots, word_to_vec), epochs=13, steps_per_epoch=n.shape[0])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (1, None, 300)       0                                            
__________________________________________________________________________________________________
Encoder (GRU)                   [(1, None, 100), (No 120300      Input[0][0]                      
__________________________________________________________________________________________________
Decoder (GRU)                   (1, None, 100)       60300       Encoder[0][0]                    
                                                                 Encoder[0][1]                    
__________________________________________________________________________________________________
intent_classifier (Dense)       (None, 8)            808         Encoder[0][1]                    
__________

48/48 [==============================] - 0s 10ms/step - loss: 0.0292 - intent_classifier_loss: 0.0227 - slots_tagger_loss: 0.0065 - intent_classifier_acc: 1.0000 - slots_tagger_acc: 1.0000
Epoch 12/13
23/48 [=============>................] - ETA: 0s - loss: 0.0318 - intent_classifier_loss: 0.0141 - slots_tagger_loss: 0.0178 - intent_classifier_acc: 1.0000 - slots_tagger_acc: 0.9913  ['i', 'have', 'to', 'do', 'chapter', '3']
['o', 'o', 'o', 'o', 'task_name', 'task_name', 'task_deadline']
['i', 'have', 'to', 'study', 'chapter', '1']
['o', 'o', 'o', 'o', 'task_name', 'task_name', 'task_deadline']
48/48 [==============================] - 1s 12ms/step - loss: 0.0254 - intent_classifier_loss: 0.0150 - slots_tagger_loss: 0.0105 - intent_classifier_acc: 1.0000 - slots_tagger_acc: 0.9958
Epoch 13/13
22/48 [============>.................] - ETA: 0s - loss: 0.0632 - intent_classifier_loss: 0.0124 - slots_tagger_loss: 0.0508 - intent_classifier_acc: 1.0000 - slots_tagger_acc: 0.9818['i', 'have', '

In [ ]:
while True:
    sent=input("Enter\n")
    if sent == "-1":
        break
    tokens=sent.split()
    ip=np.zeros((1,len(tokens),300)) 
    for i, token in enumerate(tokens):
        if token in word_to_vec:
            ip[0,i]=word_to_vec[token]
        else:
            ip[0,i]=[0]*300
    pred=model.predict(ip)
    intents_pred=pred[0]
    intents_pred=list(np.squeeze(intents_pred))
    print(intents_pred)
    print("Intent:", intents[intents_pred.index(max(intents_pred))])
    slots_pred=pred[1]
    slots_pred=np.squeeze(slots_pred)
    for pred in slots_pred:
        pred=list(pred)        
        print(slots[pred.index(max(pred))], end=" ")

Enter
create my schedule
[0.0020606716, 0.0015212797, 0.00042264396, 0.22570257, 0.7619925, 0.006033205, 0.0021321434, 0.00013495752]
Intent: displaySchedule
o o o Enter
create a schedule
[0.0005449532, 2.5732883e-05, 0.0012992973, 0.9807747, 0.009897022, 0.0049441475, 0.002474482, 3.9615577e-05]
Intent: createSchedule
o o o Enter
show my schedule
[0.0016059638, 0.0027767376, 9.727202e-05, 0.03524528, 0.95763975, 0.0017524217, 0.00075732416, 0.00012522226]
Intent: displaySchedule
o o o Enter
add chapter 1
[0.00016549541, 0.000660592, 0.72091603, 0.0031030257, 0.0004843531, 0.27075765, 0.0029233333, 0.0009895193]
Intent: addTask
o task_name task_name Enter
chapter 2 done
[0.006686756, 0.0008302935, 0.17399232, 0.0041141673, 0.00064135215, 0.69660395, 0.0044928864, 0.11263816]
Intent: removeTask
o task_name o 